In [ ]:
# 필요한 라이브러리를 임포트합니다.
import os  # 파일 및 디렉토리 작업을 위한 Python 내장 라이브러리
import cv2  # OpenCV 라이브러리를 사용하여 이미지 처리 작업을 수행
from PIL import Image  # 이미지를 열고 처리하기 위한 Pillow 라이브러리
import pandas as pd  # 데이터 분석 및 조작을 위한 라이브러리
import numpy as np  # 과학적 계산을 위한 다차원 배열 라이브러리

# PyTorch 및 관련 모듈을 임포트합니다.
import torch  # 딥러닝 모델을 구축하고 학습하기 위한 기본 라이브러리
import torch.nn as nn  # PyTorch의 신경망 모듈을 포함한 하위 모듈
from torch.utils.data import Dataset, DataLoader  # 데이터 로딩 및 처리 도구
from torchvision import transforms  # 이미지 데이터 전처리 및 변환 모듈
import torchvision

# 데이터 증강 및 전처리를 위한 Albumentations 라이브러리를 임포트합니다.
from tqdm import tqdm  # 진행률 표시를 위한 라이브러리
import albumentations as A  # 이미지 데이터 증강을 위한 라이브러리
from albumentations.pytorch import ToTensorV2  # Albumentations 출력을 PyTorch 텐서로 변환


# 시각화
import matplotlib.pyplot as plt

# 사용 가능한 GPU가 있다면 'cuda'를, 그렇지 않으면 'cpu'를 선택하여 디바이스를 설정합니다.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

이 코드는 확산 프로세스의 파라미터 및 스케줄을 설정하는 부분으로, 시간에 따른 베타 값을 선형적으로 변화시켜 확산 프로세스를 모델링합니다. 이 프로세스는 후속 딥러닝 모델에 사용될 수 있으며, 시간에 따라 노이즈를 적용하는 데 사용됩니다. 주어진 주석을 통해 코드의 의미와 목적을 이해하실 수 있을 것입니다.

In [ ]:
# Diffusion process parameters
T = 100  # 확산 프로세스 단계 수, 시간의 흐름을 모델링하는 데 사용됨
beta_min = 0.01  # 베타의 최소값 (노이즈에 대한 최소 가중치)

# Create a diffusion process schedule
# 확산 프로세스 스케줄 생성

# 베타 값을 시간에 따라 선형적으로 변화시킵니다.
# 초기 값은 0이며 T 단계 동안 beta_min 값에 수렴합니다.
# 이러한 베타 값은 확률적 모델에서 사용될 수 있습니다.
betas = np.linspace(0, beta_min, T)

# NumPy 배열을 PyTorch 텐서로 변환합니다.
betas = torch.tensor(betas, dtype=torch.float32)

이 코드는 간단한 생성자와 판별자 신경망을 정의합니다. 생성자는 잡음 벡터를 입력으로 받고, 시간 변수 t에 따라 잡음과 생성된 이미지를 혼합하여 결과를 생성합니다. 판별자는 이미지를 입력으로 받고, 이미지가 진짜인지 가짜인지 이진 분류 결과를 반환합니다. 이러한 생성자와 판별자는 확률적 모델에서 사용되며, 주로 이미지 생성 GANs (Generative Adversarial Networks)에서 사용됩니다. 코드 주석을 통해 각 부분의 역할과 목적을 이해하실 수 있을 것입니다. 다른 질문이나 추가 설명이 필요하면 언제든지 물어보세요.







In [ ]:
# Define a simple generator and discriminator network

# 간단한 생성자 (Generator) 신경망 정의
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc = nn.Linear(64, 784)  # 예시: 잡음 벡터를 이미지 공간으로 매핑

    def forward(self, z, t):
        # 노이즈를 생성하고, 시간 t에 따라 노이즈와 생성된 이미지를 혼합
        noise = torch.randn_like(z)  # 잡음 생성
        x = (1 - t.view(-1, 1, 1, 1)) * self.fc(z) + t.view(-1, 1, 1, 1) * noise
        return x

# 간단한 판별자 (Discriminator) 신경망 정의 (예시)
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(784, 128),  # 예시: 이미지 공간에서 이진 분류를 위한 매핑
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        # 입력 이미지를 판별하여 이진 분류 결과 반환
        return self.fc(x)

이 코드는 세그멘테이션 작업을 위한 U-Net 아키텍처를 정의합니다. U-Net은 인코더와 디코더로 구성되며, 주로 이미지 세그멘테이션 작업에서 사용됩니다. 인코더는 입력 이미지를 다운샘플링하고 특징을 추출하는 역할을 하며, 디코더는 이러한 특징을 업샘플링하여 최종 세그멘테이션 맵을 생성합니다. 주석을 통해 각 부분의 역할과 목적을 이해하실 수 있을 것입니다. 다른 질문이나 추가 설명이 필요하면 언제든지 물어보세요.

In [ ]:
# Define the U-Net architecture for segmentation

# 세그멘테이션을 위한 U-Net 아키텍처 정의
class UNet(nn.Module):
    # U-Net 아키텍처 정의 (이전과 동일)
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        # 인코더 (Encoder) 부분 정의
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 디코더 (Decoder) 부분 정의
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, out_channels, kernel_size=2, stride=2)
        )

    def forward(self, x):
        # 인코더 부분을 통과한 후 디코더 부분을 통과시켜 세그멘테이션 결과를 반환
        x1 = self.encoder(x)
        x2 = self.decoder(x1)
        return x2

이 코드는 U-Net 세그멘테이션 모델을 초기화하고, 세그멘테이션 작업을 위한 손실 함수와 옵티마이저를 정의하는 부분입니다. U-Net은 이미지 세그멘테이션 작업에서 자주 사용되는 신경망 아키텍처 중 하나이며, Cross-Entropy 손실 함수는 세그멘테이션 작업에 적합한 손실 함수 중 하나입니다. Adam 옵티마이저를 사용하여 모델을 최적화하고, 역확산 손실 함수로는 MSE 손실 함수를 사용하고 있습니다.

In [ ]:
# Initialize the U-Net segmentation model

# 예시: 클래스 수에 맞게 설정된 변수
num_classes = 10

# Input 이미지의 채널 수 (예: RGB 이미지의 경우 3 채널)
in_channels = 3

# 세그멘테이션 클래스의 수를 지정하는 변수
out_channels = num_classes

# U-Net 세그멘테이션 모델을 초기화합니다.
segmentation_model = UNet(in_channels, out_channels)

# Define segmentation loss function (e.g., cross-entropy loss)

# 세그멘테이션 손실 함수를 정의합니다. 주로 Cross-Entropy 손실 함수를 사용합니다.
segmentation_criterion = nn.CrossEntropyLoss()

# Define optimizer for segmentation model

# 세그멘테이션 모델을 최적화하기 위한 Adam 옵티마이저를 정의합니다.
# 학습률 (lr)은 0.0001로 설정되어 있습니다.
segmentation_optimizer = torch.optim.Adam(segmentation_model.parameters(), lr=0.0001)

# Define the reverse diffusion loss (MSE loss)

# 역확산 손실 함수를 정의합니다. 평균 제곱 오차 (MSE) 손실 함수를 사용합니다.
reverse_diffusion_criterion = nn.MSELoss()

이 코드는 U-Net 세그멘테이션 모델의 학습을 수행하는 루프입니다. 각 에폭(epoch)마다 데이터로더(data_loader)에서 미니배치를 가져와 U-Net 모델을 학습합니다. 확산(diffusion) 과정과 역확산(reverse diffusion) 과정이 번갈아 가면서 진행되며, 각각의 과정에서 손실을 계산하고 모델을 업데이트합니다. 학습이 완료된 후에는 최종 손실을 출력합니다.

In [ ]:
# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0  # 손실을 누적할 변수 초기화

    for step, (real_images, labels) in enumerate(data_loader):
        real_images = real_images.view(real_images.size(0), -1)
        batch_size = real_images.size(0)
        z = torch.randn(batch_size, 64)

        for t in range(T):
            current_beta = betas[t]

            # Forward Diffusion process: Add noise to the initial image

            # 초기 이미지에 노이즈를 추가하여 확산 과정 수행
            noise = torch.randn_like(real_images) * (current_beta ** 0.5)
            x_t = real_images + noise

            # Segmentation
            segmentation_output = segmentation_model(x_t)

            # Calculate segmentation loss

            # 세그멘테이션 모델의 출력과 실제 레이블 간의 손실 계산
            segmentation_loss = segmentation_criterion(segmentation_output, labels)

            # Update segmentation model

            # 세그멘테이션 모델의 가중치 업데이트
            segmentation_optimizer.zero_grad()
            segmentation_loss.backward()
            segmentation_optimizer.step()

            # Reverse Diffusion process: Remove noise from the image

            # 확산 과정을 역으로 수행하여 이미지에서 노이즈 제거
            noise = torch.randn_like(x_t) * (current_beta ** 0.5)
            x_t = x_t - noise

            # Calculate Reverse Diffusion loss (MSE loss)

            # 역확산 손실 계산 (평균 제곱 오차 손실)
            reverse_diffusion_loss = reverse_diffusion_criterion(x_t, real_images)

            # Update the segmentation model using the reverse diffusion optimizer

            # 역확산 손실을 사용하여 세그멘테이션 모델 업데이트
            segmentation_optimizer.zero_grad()  # 세그멘테이션 모델의 그래디언트 초기화
            reverse_diffusion_loss.backward()
            segmentation_optimizer.step()  # 세그멘테이션 모델의 파라미터 업데이트

        # 손실을 누적
        total_loss += segmentation_loss.item() + reverse_diffusion_loss.item()

    # 평균 손실 계산

    # 현재 에폭의 평균 손실 계산
    average_loss = total_loss / len(data_loader)

    # 일정 간격으로 손실 출력

    # 지정한 간격마다 현재 에폭의 평균 손실 출력
    if (epoch + 1) % print_interval == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {average_loss:.4f}")

# 학습 완료 후에도 손실 출력

# 학습이 완료된 후 최종 평균 손실 출력
print(f"Training completed. Final loss: {average_loss:.4f}")

이 코드는 세그멘테이션 모델을 사용하여 이미지 세그멘테이션을 수행하는 추론 루프를 나타냅니다. 주석을 통해 다음과 같은 추론 프로세스를 설명하고 있습니다:

바깥쪽 루프 (for _ in range(num_samples))는 추론을 여러 번 수행하는 데 사용됩니다. num_samples는 추론 반복 횟수를 나타냅니다.

잡음 벡터 z를 생성합니다. 이 벡터는 추론의 초기 입력으로 사용됩니다.

초기 입력 이미지 x_t를 생성합니다. 이 예시에서는 크기가 128x128이고 3개의 채널을 가지는 이미지를 사용합니다. 필요에 따라 이미지의 크기와 채널 수를 조절할 수 있습니다.

시간 스텝 T 동안 다음을 반복합니다:

현재 시간 스텝에 해당하는 베타 값(current_beta)을 가져옵니다.
초기 이미지에 노이즈를 추가한 후(x_t), 훈련된 세그멘테이션 모델을 사용하여 세그멘테이션 맵을 생성합니다.
생성된 세그멘테이션 맵을 segmentation_results 리스트에 추가합니다.

이 코드는 학습된 세그멘테이션 모델을 사용하여 이미지에 대한 세그멘테이션을 추론하고, 그 결과를 리스트에 저장하는 추론 프로세스를 나타냅니다.

In [ ]:
# Inference loop for segmentation (세그멘테이션 추론 루프)
segmentation_results = []  # 세그멘테이션 결과를 저장할 리스트

for _ in range(num_samples):
    z = torch.randn(1, 64)  # 잡음 벡터 생성

    # 초기 입력 이미지를 생성 (크기와 채널 수를 필요에 따라 조절)
    x_t = torch.zeros(1, 3, 128, 128)  # 예시에서는 3개의 채널과 128x128 크기의 이미지를 사용

    for t in range(T):
        current_beta = betas[t]  # 현재 시간 스텝에 해당하는 베타 값

        # 확산 프로세스: 초기 이미지에 노이즈 추가
        noise = torch.randn_like(x_t) * (current_beta ** 0.5)
        x_t = x_t + noise

    # 훈련된 세그멘테이션 모델을 사용하여 x_t에 대한 세그멘테이션 수행
    segmentation_map = segmentation_model(x_t)

    # 세그멘테이션 결과를 리스트에 추가
    segmentation_results.append(segmentation_map)

In [ ]:
# Visualize or analyze segmentation results

# 시각화 함수 추가
def visualize_segmentation(image, segmentation_map, title="Segmentation"):
    # 입력 이미지와 세그멘테이션 결과를 시각화하는 함수 정의
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    ax1.imshow(image)
    ax1.set_title("Input Image")

    ax2.imshow(segmentation_map, cmap="viridis")  # 적절한 컬러맵 사용
    ax2.set_title(title)

    plt.show()

# 결과 시각화
for i, segmentation_map in enumerate(segmentation_results):
    # 세그멘테이션 결과를 시각화하는 반복문
    input_image = data_loader[i][0]  # 입력 이미지를 가져옴
    visualize_segmentation(input_image, segmentation_map, title=f"Segmentation Result {i+1}")